In [1]:
import pandas as pd
import numpy as np

import os
cwd = os.getcwd()
kaggle = cwd == "/kaggle/working"

# pretrain = pd.read_parquet("pretrain.parquet")
# train = pd.read_parquet("train.parquet")
# test = pd.read_parquet("test.parquet")

pretrain = pd.read_parquet(("/kaggle/input/latsis-experiments/" if kaggle else "") + "pretrain.parquet")
train = pd.read_parquet(("/kaggle/input/latsis-experiments/" if kaggle else "") + "train.parquet")
test = pd.read_parquet(("/kaggle/input/latsis-experiments/" if kaggle else "") + "test.parquet")

#convert to string
train["text"] = train["text"].astype(str)
test["text"] = test["text"].astype(str)

#keep only the first 1000 rows
# pretrain = pretrain[:1000]
# train = train[:100]
test = test[:2500]

In [2]:
train["label"].sum()/len(train)

0.32650414937759337

In [3]:
import torch
from torch import nn
from transformers import TrainingArguments, Trainer, AutoTokenizer, XLMRobertaTokenizerFast, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup, TrainerCallback, TrainerControl, TrainingArguments
# model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=1)
# tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
# model = AutoModelForSequenceClassification.from_pretrained("microsoft/mdeberta-v3-base", num_labels=1, ignore_mismatched_sizes=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/mdeberta-v3-base")
# model = AutoModelForSequenceClassification.from_pretrained('aari1995/German_Semantic_STS_V2', num_labels=1)
# tokenizer = AutoTokenizer.from_pretrained('aari1995/German_Semantic_STS_V2')
# model = AutoModelForSequenceClassification.from_pretrained("deepset/gelectra-base", num_labels=1, ignore_mismatched_sizes=True)
# tokenizer = AutoTokenizer.from_pretrained("deepset/gelectra-base")
# model = AutoModelForSequenceClassification.from_pretrained("PM-AI/sts_paraphrase_xlm-roberta-base_de-en", num_labels=1, ignore_mismatched_sizes=True)
# tokenizer = AutoTokenizer.from_pretrained("PM-AI/sts_paraphrase_xlm-roberta-base_de-en")
# model = AutoModelForSequenceClassification.from_pretrained("intfloat/multilingual-e5-large", num_labels=1, ignore_mismatched_sizes=True)
# tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")
model = AutoModelForSequenceClassification.from_pretrained("deutsche-telekom/gbert-large-paraphrase-euclidean", num_labels=1, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained("deutsche-telekom/gbert-large-paraphrase-euclidean")
config = model.config
tokenizer.model_max_length = config.max_position_embeddings
# model.classifier.out_proj = nn.Sequential(
#     nn.Linear(config.hidden_size, 1),
#     nn.Sigmoid()
# )



print("model parameters:" + str(sum(p.numel() for p in model.parameters())))


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116_nocublaslt.dll
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.6\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116_nocublaslt.dll...


c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: C:\ProgramData\Anaconda3 did not contain ['cudart64_110.dll', 'cudart64_120.dll'] as expected! Searching further paths...
  warn(msg)
c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Program Files/NVIDIA/CUDNN/v8.x'), WindowsPath('C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/libnvvp'), WindowsPath('C:/Users/Timothe/.dotnet/tools'), WindowsPath('C:/ProgramData/Anaconda3/Library/usr/bin'), WindowsPath('E:/texlive/2022/bin/win32'), WindowsPath('C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin')}
  warn(msg)
c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


c:\ProgramData\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Timothe\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at intfloat/multilingual-e5-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model parameters:559891457


In [4]:
from torch.utils.data import Dataset
import torch
import numpy as np

max_length = 128

def encode_texts(tokenizer, texts):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    return torch.tensor(input_ids), torch.tensor(attention_masks)

pretrain_x, pretrain_attention_mask = encode_texts(tokenizer, pretrain['text'])
pretrain_y = torch.tensor(np.array(pretrain['label'].tolist()), dtype=torch.float32)

train_x, train_attention_mask = encode_texts(tokenizer, train['text'])
train_y = torch.tensor(np.array(train['label'].tolist()), dtype=torch.float32)

test_x, test_attention_mask = encode_texts(tokenizer, test['text'])
test_y = torch.tensor(np.array(test['label'].tolist()), dtype=torch.float32)

#mix of train and pretrain
mixed_x = torch.cat((train_x, pretrain_x))
mixed_attention_mask = torch.cat((train_attention_mask, pretrain_attention_mask))
mixed_y = torch.cat((train_y, pretrain_y))


class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, label):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.label = label

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'label': self.label[idx],
        }

pretrain_dataset = CustomDataset(pretrain_x, pretrain_attention_mask, pretrain_y)
train_dataset = CustomDataset(train_x, train_attention_mask, train_y)
val_dataset = CustomDataset(test_x, test_attention_mask, test_y)
mixed_dataset = CustomDataset(mixed_x, mixed_attention_mask, mixed_y)

print(train_x.shape, train_attention_mask.shape, train_y.shape, test_x.shape, test_attention_mask.shape, test_y.shape, pretrain_x.shape, pretrain_attention_mask.shape, pretrain_y.shape, mixed_x.shape, mixed_attention_mask.shape, mixed_y.shape)


torch.Size([7712, 128]) torch.Size([7712, 128]) torch.Size([7712]) torch.Size([2500, 128]) torch.Size([2500, 128]) torch.Size([2500]) torch.Size([19133, 128]) torch.Size([19133, 128]) torch.Size([19133]) torch.Size([26845, 128]) torch.Size([26845, 128]) torch.Size([26845])


In [5]:
import torch
from torch import nn
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F

model = model.cuda()
optimizer = None



criterion = BCEWithLogitsLoss()

# Training function
def train(model, lr_per_epoch, train_dataset, val_dataset):
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=4)

    for epoch in range(len(lr_per_epoch)):
        model.train()
        lr = lr_per_epoch[epoch]
        optimizer.param_groups[0]['lr'] = lr
        train_loss = 0.0

        # Training loop with tqdm
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            inputs = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            labels = batch['label'].cuda()

            outputs = model(inputs, attention_mask=attention_mask)
            logits = outputs.logits.squeeze(-1)  # Remove the last dimension

            loss = criterion(logits, labels)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
            
            train_loss += loss.item()
            
        avg_train_loss = train_loss / len(train_loader)
        

        # Validation loop
        model.eval()
        val_loss = 0.0
        all_predictions_raw = []
        all_labels = []

        # Validation loop with tqdm
        for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            inputs = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            labels = batch['label'].cuda()

            with torch.no_grad():
                outputs = model(inputs, attention_mask=attention_mask, labels=labels)
                logits = outputs.logits.squeeze(-1)
                val_loss += criterion(logits, labels).item()
                pred = F.sigmoid(logits)
                
                all_predictions_raw.append(pred.cpu())
                all_labels.append(labels.cpu())

        avg_val_loss = val_loss / len(val_loader)
        

        all_predictions_raw = torch.cat(all_predictions_raw)
        all_labels = torch.cat(all_labels)
        accuracy = accuracy_score(all_labels.numpy(), all_predictions_raw.numpy() >= 0.5)
        print(f"Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}, Accuracy: {accuracy}")
        # test different treshholds
        for tresh in np.arange(0.2, 0.75, 0.05):
            f1 = f1_score(all_labels.numpy(), all_predictions_raw.numpy() >= tresh, average='macro')
            print(f"tresh: {tresh}, f1: {f1}")

        

        # print(f"First predictions:")
        # i = 0
        # for y_pred, y in zip(all_predictions_raw, all_labels):
        #     print(f"y_pred: {y_pred.item()}, y: {y}")
        #     i += 1
        #     if i > 5:
        #         break

        print("\n")

# Train the model

# for param in model.parameters():
#     param.requires_grad = False
# for param in model.classifier.parameters():
#     param.requires_grad = True
# for i in range(-3, 0):
#     for param in model.roberta.encoder.layer[i].parameters():
#         param.requires_grad = True
# optimizer = AdamW([param for param in model.parameters() if param.requires_grad], lr=1e-5)

# train(model, [1e-9, 1e-5, 1e-5, 1e-5], mixed_dataset, val_dataset)



for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True
for i in range(-3, 0):
    for param in model.roberta.encoder.layer[i].parameters():
    # for param in model.bert.encoder.layer[i].parameters():
    # for param in model.deberta.encoder.layer[i].parameters():
    # for param in model.electra.encoder.layer[i].parameters():
        param.requires_grad = True
optimizer = AdamW([param for param in model.parameters() if param.requires_grad], lr=1e-5)

train(model, [1e-9, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 1e-6], train_dataset, val_dataset)

# Save the model
torch.save(model, '/kaggle/working/model.pt' if kaggle else 'model.pt')
torch.save(tokenizer, '/kaggle/working/tokenizer.pt' if kaggle else 'tokenizer.pt')
print("saved")

c:\ProgramData\Anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Validation Epoch 1: 100%|██████████| 625/625 [00:53<00:00, 11.74it/s]


Train Loss: 0.6680077667738392, Val Loss: 0.6586038119316101, Accuracy: 0.6604
tresh: 0.2, f1: 0.2431123221313957
tresh: 0.25, f1: 0.2431123221313957
tresh: 0.3, f1: 0.2431123221313957
tresh: 0.35, f1: 0.24443678133456628
tresh: 0.39999999999999997, f1: 0.2993631787576275
tresh: 0.44999999999999996, f1: 0.5291500142036318
tresh: 0.49999999999999994, f1: 0.45830699305892825
tresh: 0.5499999999999999, f1: 0.4037681850703553
tresh: 0.5999999999999999, f1: 0.40433643078389325
tresh: 0.6499999999999999, f1: 0.40433643078389325
tresh: 0.7, f1: 0.40433643078389325




Training Epoch 2:   4%|▍         | 82/1928 [00:09<03:39,  8.42it/s]

Training Epoch 2:   5%|▍         | 88/1928 [00:10<03:41,  8.31it/s]

In [ ]:
model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,